In [787]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [788]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [789]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [790]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [791]:
dim_fase = pd.read_sql_table('dim_fase_servicio', etl_conn) 
fecha = pd.read_sql_table('dim_fecha', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)
estado_servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [792]:
servicio = servicio[["id", "fecha_solicitud", "hora_solicitud", "cliente_id", "mensajero_id"]]
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,cliente_id,mensajero_id
0,34,2023-10-26,09:46:03,5,NaN
1,35,2023-10-26,11:18:14,5,7.0
2,36,2023-10-28,19:21:01,5,NaN
3,41,2023-11-07,09:46:09,5,NaN
4,42,2023-11-07,09:46:10,5,NaN


In [793]:
estado_servicio.head(5)

,id,fecha,hora,foto,observaciones,estado_id,servicio_id,es_prueba,foto_binary
0,1014,2024-01-29,01:13:32,foto,4 tubos,4,226,False,None
1,1484,2024-01-30,18:45:12,foto,Demora,3,79,True,None
2,2829,2024-02-06,11:34:04,foto,Compra exitosa,5,613,False,None
3,1888,2024-02-01,14:50:39,foto,Zzxzz,4,376,False,None
4,32312,2024-04-06,16:11:21,foto,No,3,7164,True,None


In [794]:
print(len(estado_servicio))

128402


In [795]:
estado_servicio.drop(columns=["foto","observaciones","es_prueba","foto_binary"], inplace=True)
estado_servicio

,id,fecha,hora,estado_id,servicio_id
0,1014,2024-01-29,01:13:32,4,226
1,1484,2024-01-30,18:45:12,3,79
2,2829,2024-02-06,11:34:04,5,613
3,1888,2024-02-01,14:50:39,4,376
4,32312,2024-04-06,16:11:21,3,7164
...,...,...,...,...,...
128397,128805,2024-08-31,15:01:03,2,28467
128398,128806,2024-08-31,15:03:42,2,28466
128399,128807,2024-08-31,15:04:26,4,28466
128400,128808,2024-08-31,15:13:47,3,28467


## Transformation

### Hour Process

In [796]:
estado_servicio['hora'] = estado_servicio['hora'].astype(str)
# Elimina los milisegundos dividiendo en el punto y tomando solo la parte de HH:MM:SS
estado_servicio['hora'] = estado_servicio['hora'].str.split('.').str[0]
# Convierte las horas a datetime
estado_servicio['hora'] = pd.to_datetime(estado_servicio['hora'], format='%H:%M:%S', errors='coerce')
estado_servicio.head(10)


,id,fecha,hora,estado_id,servicio_id
0,1014,2024-01-29,1900-01-01 01:13:32,4,226
1,1484,2024-01-30,1900-01-01 18:45:12,3,79
2,2829,2024-02-06,1900-01-01 11:34:04,5,613
3,1888,2024-02-01,1900-01-01 14:50:39,4,376
4,32312,2024-04-06,1900-01-01 16:11:21,3,7164
5,2426,2024-02-04,1900-01-01 11:15:40,3,377
6,48803,2024-05-03,1900-01-01 06:11:39,3,10910
7,3323,2024-02-07,1900-01-01 16:26:03,2,746
8,3886,2024-02-09,1900-01-01 13:25:05,6,842
9,4211,2024-02-10,1900-01-01 13:46:43,2,930


### Date Process

In [797]:
estado_servicio = pd.merge(estado_servicio, fecha[["date", "key_fecha"]], left_on="fecha", right_on="date", how="left")
estado_servicio

,id,fecha,hora,estado_id,servicio_id,date,key_fecha
0,1014,2024-01-29,1900-01-01 01:13:32,4,226,2024-01-29,385
1,1484,2024-01-30,1900-01-01 18:45:12,3,79,2024-01-30,386
2,2829,2024-02-06,1900-01-01 11:34:04,5,613,2024-02-06,393
3,1888,2024-02-01,1900-01-01 14:50:39,4,376,2024-02-01,388
4,32312,2024-04-06,1900-01-01 16:11:21,3,7164,2024-04-06,453
...,...,...,...,...,...,...,...
128397,128805,2024-08-31,1900-01-01 15:01:03,2,28467,2024-08-31,600
128398,128806,2024-08-31,1900-01-01 15:03:42,2,28466,2024-08-31,600
128399,128807,2024-08-31,1900-01-01 15:04:26,4,28466,2024-08-31,600
128400,128808,2024-08-31,1900-01-01 15:13:47,3,28467,2024-08-31,600


In [798]:
len(servicio)

28430

### Phase Process

In [799]:
# dim_fase.rename(columns={'id':'dim_estado_id'}, inplace=True)
# estado_servicio = pd.merge(estado_servicio, dim_fase[["dim_estado_id", "key_fase_servicio"]], left_on="estado_id", right_on="dim_estado_id", how="left")
# estado_servicio

In [800]:
fase_iniciado = estado_servicio[estado_servicio['estado_id'] == 1]
fase_mensajero_asignado = estado_servicio[estado_servicio['estado_id'] == 2]
fase_recogido_mensajero = estado_servicio[estado_servicio['estado_id'] == 4]
fase_entregado_destino = estado_servicio[estado_servicio['estado_id'] == 5]
fase_terminado = estado_servicio[estado_servicio['estado_id'] == 6]

In [801]:
fase_iniciado.rename(columns={'fecha':'fecha_inicio', 'hora':'hora_inicio', 'key_fecha':'key_fecha_inicio'}, inplace=True)
fase_iniciado = fase_iniciado[['servicio_id','fecha_inicio','hora_inicio','key_fecha_inicio']]
fase_iniciado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado'}, inplace=True)
fase_mensajero_asignado = fase_mensajero_asignado[['servicio_id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado']]
fase_mensajero_asignado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajero'}, inplace=True)
fase_recogido_mensajero = fase_recogido_mensajero[['servicio_id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero']]
fase_recogido_mensajero.rename(columns={'servicio_id':'id'}, inplace=True)

fase_entregado_destino.rename(columns={'fecha':'fecha_entregado_destino', 'hora':'hora_entregado_destino','key_fecha':'key_fecha_entregado_destino'}, inplace=True)
fase_entregado_destino = fase_entregado_destino[['servicio_id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino']]
fase_entregado_destino.rename(columns={'servicio_id':'id'}, inplace=True)

fase_terminado.rename(columns={'fecha':'fecha_terminado', 'hora':'hora_terminado','key_fecha':'key_fecha_terminado'}, inplace=True)
fase_terminado = fase_terminado[['servicio_id','fecha_terminado','hora_terminado','key_fecha_terminado']]
fase_terminado.rename(columns={'servicio_id':'id'}, inplace=True)

C:\Users\nicol\AppData\Local\Temp\ipykernel_1332\256166447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_iniciado.rename(columns={'fecha':'fecha_inicio', 'hora':'hora_inicio', 'key_fecha':'key_fecha_inicio'}, inplace=True)
C:\Users\nicol\AppData\Local\Temp\ipykernel_1332\256166447.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado'}, inplace=True)
C:\Users\nicol\AppData\Local\Temp\ipykernel_1332\256166447.py:9: SettingWithCopyWarning: 
A value is trying to b

In [802]:
print(fase_iniciado['id'].duplicated().sum())  # Muestra cuántos duplicados hay
print(fase_mensajero_asignado['id'].duplicated().sum())
print(fase_recogido_mensajero['id'].duplicated().sum())
print(fase_entregado_destino['id'].duplicated().sum())
print(fase_terminado['id'].duplicated().sum())

1198
2547
526
383
157


In [803]:
fase_iniciado = fase_iniciado.drop_duplicates(subset='id')
fase_mensajero_asignado = fase_mensajero_asignado.drop_duplicates(subset='id')
fase_recogido_mensajero = fase_recogido_mensajero.drop_duplicates(subset='id')
fase_entregado_destino = fase_entregado_destino.drop_duplicates(subset='id')
fase_terminado = fase_terminado.drop_duplicates(subset='id')

In [804]:
len(fase_entregado_destino)

26952

In [805]:
servicio = pd.merge(servicio, fase_iniciado[['id','fecha_inicio','hora_inicio','key_fecha_inicio']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_mensajero_asignado[['id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_recogido_mensajero[['id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_entregado_destino[['id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino']], left_on="id", right_on="id", how="left")
servicio = pd.merge(servicio, fase_terminado[['id','fecha_terminado','hora_terminado','key_fecha_terminado']], left_on="id", right_on="id", how="left")
servicio

,id,fecha_solicitud,hora_solicitud,cliente_id,mensajero_id,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado
0,34,2023-10-26,09:46:03,5,NaN,2023-10-26,1900-01-01 09:46:03,290.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
1,35,2023-10-26,11:18:14,5,7.0,2023-10-26,1900-01-01 11:18:14,290.0,2023-10-28,1900-01-01 14:43:08,292.0,2023-10-28,1900-01-01 19:45:18,292.0,2023-12-07,1900-01-01 01:25:34,332.0,NaT,NaT,NaN
2,36,2023-10-28,19:21:01,5,NaN,2023-10-28,1900-01-01 19:21:01,292.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
3,41,2023-11-07,09:46:09,5,NaN,2023-11-07,1900-01-01 09:46:09,302.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
4,42,2023-11-07,09:46:10,5,NaN,2023-11-07,1900-01-01 09:46:10,302.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28425,28267,2024-08-30,10:52:02,11,8.0,2024-08-30,1900-01-01 10:52:02,599.0,2024-08-30,1900-01-01 11:00:55,599.0,2024-08-30,1900-01-01 11:33:40,599.0,2024-08-30,1900-01-01 11:47:52,599.0,NaT,NaT,NaN
28426,28363,2024-08-30,16:02:24,11,22.0,2024-08-30,1900-01-01 16:02:24,599.0,2024-08-30,1900-01-01 16:10:22,599.0,2024-08-30,1900-01-01 16:16:13,599.0,2024-08-30,1900-01-01 17:29:59,599.0,NaT,NaT,NaN
28427,28403,2024-08-31,07:52:50,5,30.0,2024-08-31,1900-01-01 07:52:50,600.0,2024-08-31,1900-01-01 07:56:42,600.0,2024-08-31,1900-01-01 08:48:57,600.0,2024-08-31,1900-01-01 09:27:12,600.0,NaT,NaT,NaN
28428,28438,2024-08-31,10:42:03,11,39.0,2024-08-31,1900-01-01 10:42:03,600.0,2024-08-31,1900-01-01 10:43:35,600.0,2024-08-31,1900-01-01 11:05:36,600.0,2024-08-31,1900-01-01 11:25:37,600.0,2024-08-31,1900-01-01 11:26:14,600.0


In [806]:
print(servicio)

          id fecha_solicitud hora_solicitud  cliente_id  mensajero_id  \
0         34      2023-10-26       09:46:03           5           NaN   
1         35      2023-10-26       11:18:14           5           7.0   
2         36      2023-10-28       19:21:01           5           NaN   
3         41      2023-11-07       09:46:09           5           NaN   
4         42      2023-11-07       09:46:10           5           NaN   
...      ...             ...            ...         ...           ...   
28425  28267      2024-08-30       10:52:02          11           8.0   
28426  28363      2024-08-30       16:02:24          11          22.0   
28427  28403      2024-08-31       07:52:50           5          30.0   
28428  28438      2024-08-31       10:42:03          11          39.0   
28429  28437      2024-08-31       10:40:33          11          41.0   

      fecha_inicio         hora_inicio  key_fecha_inicio  \
0       2023-10-26 1900-01-01 09:46:03             290.0   
1  

### Calculo días

In [807]:
servicio['dias_mensajero_asignado'] = (servicio['fecha_mensajero_asignado'] - servicio['fecha_inicio']).dt.days.fillna(0)
servicio['dias_recogido_mensajero'] = (servicio['fecha_recogido_mensajero'] - servicio['fecha_mensajero_asignado']).dt.days.fillna(0)
servicio['dias_entregado_destino'] = (servicio['fecha_entregado_destino'] - servicio['fecha_recogido_mensajero']).dt.days.fillna(0)
servicio['dias_terminado'] = (servicio['fecha_terminado'] - servicio['fecha_entregado_destino']).dt.days.fillna(0)
# Calcular el promedio general de entrega en días y redondear a 2 decimales
servicio['promedio_entrega_general'] = (
    (servicio['dias_mensajero_asignado'] + 
     servicio['dias_recogido_mensajero'] + 
     servicio['dias_entregado_destino'] + 
     servicio['dias_terminado']) / 4
).round(2)
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,cliente_id,mensajero_id,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,...,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general
0,34,2023-10-26,09:46:03,5,NaN,2023-10-26,1900-01-01 09:46:03,290.0,NaT,NaT,...,NaT,NaN,NaT,NaT,NaN,0.0,0.0,0.0,0.0,0.0
1,35,2023-10-26,11:18:14,5,7.0,2023-10-26,1900-01-01 11:18:14,290.0,2023-10-28,1900-01-01 14:43:08,...,1900-01-01 01:25:34,332.0,NaT,NaT,NaN,2.0,0.0,40.0,0.0,10.5
2,36,2023-10-28,19:21:01,5,NaN,2023-10-28,1900-01-01 19:21:01,292.0,NaT,NaT,...,NaT,NaN,NaT,NaT,NaN,0.0,0.0,0.0,0.0,0.0
3,41,2023-11-07,09:46:09,5,NaN,2023-11-07,1900-01-01 09:46:09,302.0,NaT,NaT,...,NaT,NaN,NaT,NaT,NaN,0.0,0.0,0.0,0.0,0.0
4,42,2023-11-07,09:46:10,5,NaN,2023-11-07,1900-01-01 09:46:10,302.0,NaT,NaT,...,NaT,NaN,NaT,NaT,NaN,0.0,0.0,0.0,0.0,0.0


### Hora

In [808]:
# Calcular diferencias en horas, redondear a 2 decimales, llenar valores nulos con 0 y tomar valor absoluto
servicio['horas_mensajero_asignado'] = ((servicio['hora_mensajero_asignado'] - servicio['hora_inicio']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_recogido_mensajero'] = ((servicio['hora_recogido_mensajero'] - servicio['hora_mensajero_asignado']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_entregado_destino'] = ((servicio['hora_entregado_destino'] - servicio['hora_recogido_mensajero']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio['horas_terminado'] = ((servicio['hora_terminado'] - servicio['hora_entregado_destino']).dt.total_seconds() / 3600).round(2).fillna(0).abs()

In [809]:
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,cliente_id,mensajero_id,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,...,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado
0,34,2023-10-26,09:46:03,5,NaN,2023-10-26,1900-01-01 09:46:03,290.0,NaT,NaT,...,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
1,35,2023-10-26,11:18:14,5,7.0,2023-10-26,1900-01-01 11:18:14,290.0,2023-10-28,1900-01-01 14:43:08,...,NaN,2.0,0.0,40.0,0.0,10.5,3.42,5.04,18.33,0.0
2,36,2023-10-28,19:21:01,5,NaN,2023-10-28,1900-01-01 19:21:01,292.0,NaT,NaT,...,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
3,41,2023-11-07,09:46:09,5,NaN,2023-11-07,1900-01-01 09:46:09,302.0,NaT,NaT,...,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
4,42,2023-11-07,09:46:10,5,NaN,2023-11-07,1900-01-01 09:46:10,302.0,NaT,NaT,...,NaN,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0


In [810]:
servicio.fillna(-1, inplace=True)
total_nan = servicio.isna().sum()
print(total_nan)

C:\Users\nicol\AppData\Local\Temp\ipykernel_1332\581775963.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  servicio.fillna(-1, inplace=True)


id                              0
fecha_solicitud                 0
hora_solicitud                  0
cliente_id                      0
mensajero_id                    0
fecha_inicio                    0
hora_inicio                     0
key_fecha_inicio                0
fecha_mensajero_asignado        0
hora_mensajero_asignado         0
key_fecha_mensajero_asignado    0
fecha_recogido_mensajero        0
hora_recogido_mensajero         0
key_fecha_recogido_mensajero    0
fecha_entregado_destino         0
hora_entregado_destino          0
key_fecha_entregado_destino     0
fecha_terminado                 0
hora_terminado                  0
key_fecha_terminado             0
dias_mensajero_asignado         0
dias_recogido_mensajero         0
dias_entregado_destino          0
dias_terminado                  0
promedio_entrega_general        0
horas_mensajero_asignado        0
horas_recogido_mensajero        0
horas_entregado_destino         0
horas_terminado                 0
dtype: int64


## calculo tiempo por fase_servicio

In [811]:
# # Ordenar por 'servicio_id' y 'fecha'
# estado_servicio = estado_servicio.sort_values(by=['servicio_id', 'fecha'])

# # Asegurarse de que la columna 'hora' esté en formato timedelta
# estado_servicio['hora'] = pd.to_timedelta(estado_servicio['hora'].astype(str))

# # Calcular la diferencia en días entre fases consecutivas dentro de cada servicio
# estado_servicio['dias_de_demora'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: x.diff().dt.days)

# # asignar el resultado a la columna directamente para evitar errores jeje
# estado_servicio['dias_de_demora'] = estado_servicio['dias_de_demora'].fillna(0)

# # Calcular la diferencia en horas entre fases consecutivas dentro de cada servicio
# estado_servicio['hora_de_demora'] = estado_servicio.groupby('servicio_id')['hora'].transform(lambda x: x.diff())

# # Corregir las horas negativas: Si la hora actual es menor que la anterior, agregar 24 horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: x if x >= pd.Timedelta(0) else x + pd.Timedelta(days=1))

# # Convertir la diferencia en horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# # Lo mismo para la columna 'hora_de_demora'
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].fillna(0)

# # Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
# estado_servicio['hora'] = estado_servicio['hora'].dt.total_seconds() / 3600
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].apply(lambda x: round(x, 2))  # Redondear a 2 decimales


# estado_servicio.head(20)


## Calculo atributo tiempo promedio general

In [812]:
# # Convertir 'hora_de_demora' a timedelta si aún no está en el formato adecuado
# estado_servicio['hora_de_demora'] = pd.to_timedelta(estado_servicio['hora_de_demora'], unit='h')

# # Calcular el tiempo total en días considerando solo las fechas, con dos decimales
# estado_servicio['tiempo_total'] = estado_servicio.groupby('servicio_id')['fecha'].transform(lambda x: (x.max() - x.min()).days).round(2)

# # Calcular el tiempo total en horas sumando las horas de demora de cada fase por servicio
# estado_servicio['tiempo_total_horas'] = estado_servicio.groupby('servicio_id')['hora_de_demora'].transform('sum').dt.total_seconds() / 3600
# estado_servicio['tiempo_total_horas'] = estado_servicio['tiempo_total_horas'].round(2)

# # Calcular el número de fases por servicio
# estado_servicio['numero_fases'] = estado_servicio.groupby('servicio_id')['fecha'].transform('count')

# # Calcular el tiempo promedio por fase en días (dividiendo el tiempo total de días entre el número de fases)
# estado_servicio['tiempo_promedio_dias'] = (estado_servicio['tiempo_total'] / estado_servicio['numero_fases']).round(2)

# # Calcular el tiempo promedio en horas por fase con dos decimales (dividiendo el tiempo total de horas entre el número de fases)
# estado_servicio['tiempo_promedio_horas'] = (estado_servicio['tiempo_total_horas'] / estado_servicio['numero_fases']).round(2)

# # Convertir las columnas 'hora' y 'hora_de_demora' a su formato de horas
# estado_servicio['hora_de_demora'] = estado_servicio['hora_de_demora'].dt.total_seconds() / 3600

# # Mostrar los primeros 10 resultados
# estado_servicio.head(10) 


## Eliminar columnas no necesarias

In [813]:
servicio.drop(columns=['id','fecha_solicitud','hora_solicitud','fecha_inicio','hora_inicio','fecha_mensajero_asignado','hora_mensajero_asignado','fecha_recogido_mensajero','hora_recogido_mensajero','fecha_entregado_destino','hora_entregado_destino','fecha_terminado','hora_terminado'], inplace=True)
servicio.head(10)

,cliente_id,mensajero_id,key_fecha_inicio,key_fecha_mensajero_asignado,key_fecha_recogido_mensajero,key_fecha_entregado_destino,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado
0,5,-1.0,290.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1,5,7.0,290.0,292.0,292.0,332.0,-1.0,2.0,0.0,40.0,0.0,10.50,3.42,5.04,18.33,0.0
2,5,-1.0,292.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
3,5,-1.0,302.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
4,5,-1.0,302.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
5,5,-1.0,302.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
6,4,12.0,304.0,304.0,304.0,304.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.05,5.02,0.30,0.0
7,5,12.0,304.0,304.0,391.0,391.0,-1.0,0.0,87.0,0.0,0.0,21.75,0.32,0.74,0.01,0.0
8,5,12.0,306.0,306.0,306.0,306.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.04,5.02,0.04,0.0
9,5,-1.0,303.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0


In [814]:
len(servicio)

28430

## Load

In [815]:
servicio.to_sql("hecho_accumulating_snapshot", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

430